# Automated Project: Planning, Evaluation, Estimation

In [1]:
%pip install crewai crewai-tools

  Using cached crewai-0.108.0-py3-none-any.whl.metadata (33 kB)
  Using cached crewai_tools-0.38.1-py3-none-any.whl.metadata (5.5 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached auth0_python-4.8.1-py3-none-any.whl.metadata (9.0 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached instructor-1.7.7-py3-none-any.whl.metadata (22 kB)
  Using cached json_repair-0.39.1-py3-none-any.whl.metadata (11 kB)
  Using cached json5-0.10.0-py3-none-any.whl.metadata (34 kB)
  Using cached jsonref-1.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached litellm-1.60.2-py3-none-any.whl.metadata (36 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached pdfplumber-0.11.5-py3-none-any.whl.metadata (42 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cache

In [3]:
import os
import yaml
import onnxruntime
from crewai import Agent, Task, Crew